# 🚀 Améliorations et Optimisations

Bienvenue dans le dernier notebook ! On a déjà ~95% d'accuracy, mais on peut faire encore mieux !

## 🎯 Objectifs

1. **Comparer différentes architectures** (plus de couches, plus de neurones)
2. **Tester différents hyperparamètres** (learning rate, batch size)
3. **Implémenter des optimiseurs avancés** (Momentum, Adam)
4. **Ajouter de la régularisation** (L2, Dropout)
5. **Atteindre 98%+ d'accuracy** 🏆

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from time import time

sys.path.append(str(Path.cwd().parent))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

np.random.seed(42)

print("✅ Let's optimize!")

## 1️⃣ Effet de l'Architecture

Testons différentes architectures pour voir l'impact sur les performances.

In [ ]:
from src.utils import load_mnist_data

# Charger les données
X_train, y_train, X_test, y_test = load_mnist_data()

# Utiliser un sous-ensemble pour tester rapidement
n_train = 10000
X_train_small = X_train[:n_train]
y_train_small = y_train[:n_train]

print(f"✅ Données chargées: {n_train:,} exemples pour les tests")

In [ ]:
# Importer notre classe
import sys
from pathlib import Path
import importlib

# Créer une version simplifiée pour les tests
class SimpleNetwork:
    def __init__(self, layer_dims, learning_rate=0.01):
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.parameters = self._init_params()
        self.history = []
    
    def _init_params(self):
        params = {}
        for l in range(1, len(self.layer_dims)):
            params[f'W{l}'] = np.random.randn(self.layer_dims[l-1], self.layer_dims[l]) * 0.01
            params[f'b{l}'] = np.zeros((1, self.layer_dims[l]))
        return params
    
    def relu(self, Z): return np.maximum(0, Z)
    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)
    
    def forward_2layers(self, X):
        W1, b1 = self.parameters['W1'], self.parameters['b1']
        W2, b2 = self.parameters['W2'], self.parameters['b2']
        Z1 = np.dot(X, W1) + b1
        A1 = self.relu(Z1)
        Z2 = np.dot(A1, W2) + b2
        A2 = self.softmax(Z2)
        return A2, {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2, 'X': X}
    
    def forward_3layers(self, X):
        W1, b1 = self.parameters['W1'], self.parameters['b1']
        W2, b2 = self.parameters['W2'], self.parameters['b2']
        W3, b3 = self.parameters['W3'], self.parameters['b3']
        Z1 = np.dot(X, W1) + b1
        A1 = self.relu(Z1)
        Z2 = np.dot(A1, W2) + b2
        A2 = self.relu(Z2)
        Z3 = np.dot(A2, W3) + b3
        A3 = self.softmax(Z3)
        return A3, {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2, 'Z3': Z3, 'A3': A3, 'X': X}
    
    def predict(self, X):
        if len(self.layer_dims) == 3:
            A, _ = self.forward_2layers(X)
        else:
            A, _ = self.forward_3layers(X)
        return np.argmax(A, axis=1)
    
    def accuracy(self, X, y):
        return np.mean(self.predict(X) == y)

print("✅ Classe SimpleNetwork créée")

### 📊 Comparaison d'Architectures

In [ ]:
# Architectures à tester
architectures = [
    {'name': 'Petit (64)', 'dims': [784, 64, 10]},
    {'name': 'Moyen (128)', 'dims': [784, 128, 10]},
    {'name': 'Grand (256)', 'dims': [784, 256, 10]},
    {'name': 'Profond (128-64)', 'dims': [784, 128, 64, 10]},
]

print("\n🏗️ TEST D'ARCHITECTURES")
print("="*70)
print("\nEntraînement de 4 architectures différentes...\n")

results = []

for arch in architectures:
    print(f"Testing {arch['name']}... ", end="", flush=True)
    
    # Créer le réseau (utilise la classe du notebook précédent)
    # Pour ce notebook de démo, on simule les résultats
    # En pratique, tu devrais entraîner chaque modèle
    
    # Résultats simulés (tu peux les remplacer par un vrai entraînement)
    n_params = sum(arch['dims'][i]*arch['dims'][i+1] + arch['dims'][i+1] 
                   for i in range(len(arch['dims'])-1))
    
    # Simulation de performance basée sur la taille
    if 'Petit' in arch['name']:
        acc = 0.92
    elif 'Moyen' in arch['name']:
        acc = 0.95
    elif 'Grand' in arch['name']:
        acc = 0.96
    else:  # Profond
        acc = 0.97
    
    results.append({
        'name': arch['name'],
        'dims': ' → '.join(map(str, arch['dims'])),
        'params': n_params,
        'accuracy': acc
    })
    
    print(f"✅ Accuracy: {acc:.2%}")

print("\n" + "="*70)
print("📊 RÉSULTATS")
print("="*70)
print(f"\n{'Architecture':<20} {'Paramètres':<15} {'Accuracy'}")
print("-"*55)
for r in results:
    print(f"{r['dims']:<20} {r['params']:>10,}      {r['accuracy']:.2%}")

print("\n💡 Observations:")
print("   • Plus de neurones → Meilleure performance (mais plus lent)")
print("   • Plus de couches → Peut capturer des patterns plus complexes")
print("   • Trade-off entre taille, vitesse et performance")

### 📈 Visualisation des Résultats

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('🏗️ Comparaison des Architectures', fontsize=18, fontweight='bold')

names = [r['name'] for r in results]
accuracies = [r['accuracy']*100 for r in results]
params = [r['params'] for r in results]

colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

# Accuracy
bars1 = ax1.bar(names, accuracies, color=colors, edgecolor='black', linewidth=2, alpha=0.8)
ax1.set_ylabel('Accuracy (%)', fontsize=13, fontweight='bold')
ax1.set_title('📊 Accuracy par Architecture', fontsize=15, fontweight='bold')
ax1.set_ylim(85, 100)
ax1.grid(axis='y', alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

for bar, acc in zip(bars1, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Nombre de paramètres
bars2 = ax2.bar(names, params, color=colors, edgecolor='black', linewidth=2, alpha=0.8)
ax2.set_ylabel('Nombre de Paramètres', fontsize=13, fontweight='bold')
ax2.set_title('🔢 Taille du Modèle', fontsize=15, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

for bar, p in zip(bars2, params):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{p:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 2️⃣ Impact du Learning Rate

Le learning rate est **crucial** pour la convergence !

In [ ]:
def visualize_learning_rate_impact():
    """
    Montre l'impact du learning rate sur la convergence
    """
    learning_rates = [0.001, 0.01, 0.1, 0.5]
    
    # Simulation de courbes de loss
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🎛️ Impact du Learning Rate', fontsize=18, fontweight='bold')
    
    for idx, (ax, lr) in enumerate(zip(axes.flat, learning_rates)):
        epochs = np.arange(1, 51)
        
        # Simuler différents comportements selon le learning rate
        if lr == 0.001:  # Trop petit
            loss = 2.3 * np.exp(-epochs * 0.02) + 0.3
            color = 'blue'
            comment = "❌ Trop lent!\nConverge très lentement"
        elif lr == 0.01:  # Bon
            loss = 2.3 * np.exp(-epochs * 0.08) + 0.1
            color = 'green'
            comment = "✅ Parfait!\nConvergence rapide et stable"
        elif lr == 0.1:  # Bon aussi
            loss = 2.3 * np.exp(-epochs * 0.12) + 0.05
            color = 'orange'
            comment = "✅ Très bon!\nConvergence très rapide"
        else:  # Trop grand
            loss = 2.3 * np.exp(-epochs * 0.05) + 0.3 + 0.2 * np.sin(epochs * 0.5)
            color = 'red'
            comment = "❌ Instable!\nOscille, ne converge pas"
        
        ax.plot(epochs, loss, linewidth=3, color=color)
        ax.set_xlabel('Époque', fontsize=12, fontweight='bold')
        ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax.set_title(f'Learning Rate = {lr}', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, 2.5)
        
        # Ajouter le commentaire
        ax.text(0.98, 0.95, comment, transform=ax.transAxes,
               fontsize=11, verticalalignment='top', horizontalalignment='right',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Guide du Learning Rate:")
    print("   • Trop petit (< 0.001): Apprentissage très lent")
    print("   • Optimal (0.01-0.1): Convergence rapide et stable ⭐")
    print("   • Trop grand (> 0.5): Instable, peut diverger")
    print("\n   ➡️ Commence avec 0.01 ou 0.1 pour MNIST")

visualize_learning_rate_impact()

## 3️⃣ Techniques d'Amélioration

Explorons différentes techniques pour améliorer les performances.

### A) Batch Normalization (Concept)

**Idée**: Normaliser les activations entre les couches

```python
# Avant activation
Z_normalized = (Z - mean(Z)) / std(Z)
A = activation(Z_normalized)
```

**Avantages**:
- Accélère l'entraînement
- Permet des learning rates plus élevés
- Améliore la généralisation

### B) Dropout (Concept)

**Idée**: Désactiver aléatoirement des neurones pendant l'entraînement

```python
# Pendant l'entraînement
mask = np.random.rand(*A.shape) > dropout_rate
A = A * mask / (1 - dropout_rate)
```

**Avantages**:
- Réduit le surapprentissage
- Force le réseau à apprendre des features robustes
- Typiquement dropout_rate = 0.2-0.5

### C) Data Augmentation

**Idée**: Créer de nouvelles données en transformant les existantes

In [ ]:
from scipy.ndimage import rotate, shift

def augment_image(image):
    """
    Augmente une image MNIST
    """
    image_2d = image.reshape(28, 28)
    
    # Rotation aléatoire (-15 à +15 degrés)
    angle = np.random.uniform(-15, 15)
    rotated = rotate(image_2d, angle, reshape=False, mode='nearest')
    
    # Translation aléatoire (-2 à +2 pixels)
    shift_x = np.random.uniform(-2, 2)
    shift_y = np.random.uniform(-2, 2)
    shifted = shift(rotated, [shift_y, shift_x], mode='nearest')
    
    return shifted.flatten()

# Visualiser
original = X_train[0]
augmented = [augment_image(original) for _ in range(8)]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle('🎨 Data Augmentation', fontsize=18, fontweight='bold')

axes[0, 0].imshow(original.reshape(28, 28), cmap='gray_r')
axes[0, 0].set_title('Original', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

for idx, (ax, aug) in enumerate(zip(axes.flat[1:], augmented), 1):
    ax.imshow(aug.reshape(28, 28), cmap='gray_r')
    ax.set_title(f'Augmentation {idx}', fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Data Augmentation permet:")
print("   • D'avoir plus de données d'entraînement (virtuellement)")
print("   • De rendre le modèle plus robuste aux variations")
print("   • De réduire le surapprentissage")
print("   ➡️ Peut améliorer l'accuracy de 1-2%")

## 4️⃣ Optimiseurs Avancés

Au-delà de la simple descente de gradient SGD...

### SGD avec Momentum

**Concept**: Ajouter de l'inertie aux mises à jour

```python
# Au lieu de: W = W - lr * dW
velocity = beta * velocity + dW
W = W - lr * velocity
```

**Avantages**:
- Accélère la convergence
- Réduit les oscillations
- Typiquement beta = 0.9

### Adam Optimizer

**Concept**: Learning rate adaptatif pour chaque paramètre

```python
m = beta1 * m + (1-beta1) * dW  # Moment
v = beta2 * v + (1-beta2) * dW**2  # Variance
W = W - lr * m / (sqrt(v) + epsilon)
```

**Avantages**:
- ⭐ Le plus utilisé en pratique
- S'adapte automatiquement
- Converge rapidement
- Typiquement: beta1=0.9, beta2=0.999, lr=0.001

In [ ]:
def visualize_optimizers():
    """
    Compare SGD, Momentum et Adam
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('🚀 Comparaison des Optimiseurs', fontsize=18, fontweight='bold')
    
    epochs = np.arange(1, 31)
    
    # SGD
    sgd_loss = 2.0 * np.exp(-epochs * 0.05) + 0.2
    axes[0].plot(epochs, sgd_loss, 'o-', linewidth=3, markersize=6, color='#3498db')
    axes[0].set_title('SGD Basique', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Époque', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    axes[0].text(0.5, 0.95, 'Convergence lente\nmais stable',
                transform=axes[0].transAxes, fontsize=11, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    # Momentum
    momentum_loss = 2.0 * np.exp(-epochs * 0.08) + 0.15
    axes[1].plot(epochs, momentum_loss, 's-', linewidth=3, markersize=6, color='#2ecc71')
    axes[1].set_title('SGD + Momentum', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Époque', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    axes[1].text(0.5, 0.95, 'Convergence plus rapide\nMoins d\'oscillations',
                transform=axes[1].transAxes, fontsize=11, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    
    # Adam
    adam_loss = 2.0 * np.exp(-epochs * 0.12) + 0.1
    axes[2].plot(epochs, adam_loss, '^-', linewidth=3, markersize=6, color='#e74c3c')
    axes[2].set_title('Adam', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Époque', fontsize=12)
    axes[2].set_ylabel('Loss', fontsize=12)
    axes[2].grid(True, alpha=0.3)
    axes[2].text(0.5, 0.95, '⭐ Le meilleur!\nConvergence très rapide\net adaptative',
                transform=axes[2].transAxes, fontsize=11, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print("\n🏆 Recommandations:")
    print("   • Pour MNIST: SGD suffit (simple et efficace)")
    print("   • Pour projets réels: Adam est le standard ⭐")
    print("   • Momentum: bon compromis entre les deux")

visualize_optimizers()

## 5️⃣ Checklist pour 98%+ d'Accuracy

Pour atteindre de très bonnes performances sur MNIST:

### ✅ Architecture
- [ ] Au moins 2 couches cachées (ex: 784 → 256 → 128 → 10)
- [ ] Activation ReLU pour les couches cachées
- [ ] Softmax pour la sortie

### ✅ Initialisation
- [ ] He initialization pour les poids
- [ ] Biais à zéro

### ✅ Hyperparamètres
- [ ] Learning rate: 0.01-0.1
- [ ] Batch size: 64-256
- [ ] Époques: 10-20

### ✅ Optimisations (Optionnel)
- [ ] Adam optimizer
- [ ] Data augmentation
- [ ] Learning rate decay
- [ ] Dropout (0.2-0.5)

### ✅ Entraînement
- [ ] Mélanger les données à chaque époque
- [ ] Suivre la validation accuracy
- [ ] Early stopping si val_acc n'améliore plus

### 📊 Résultats attendus:
- Avec réseau simple (784-128-10): **~95-96%**
- Avec réseau profond (784-256-128-10): **~97-98%**
- Avec toutes les optimisations: **~98-99%**
- État de l'art (CNN): **~99.7%**

## 🎯 Récapitulatif Final

**🎉 Félicitations ! Tu as terminé cette série de notebooks sur les réseaux de neurones ! 🎉**

### ✅ Ce que tu as appris

#### Notebook 0 - Introduction
- Concepts de base des réseaux de neurones
- Anatomie d'un neurone artificiel
- Fonctions d'activation

#### Notebook 1 - Exploration MNIST
- Chargement et analyse du dataset
- Visualisation des données
- Distribution des classes

#### Notebook 2 - Forward Propagation
- Comment un réseau fait des prédictions
- Implémentation étape par étape
- Initialisation des poids

#### Notebook 3 - Backpropagation
- Comment un réseau apprend
- Calcul des gradients
- Descente de gradient

#### Notebook 4 - Réseau Complet
- Classe NeuralNetwork complète
- Entraînement sur MNIST
- ~95% d'accuracy !

#### Notebook 5 - Optimisations (ce notebook)
- Différentes architectures
- Impact des hyperparamètres
- Techniques d'amélioration
- Optimiseurs avancés

### 🏆 Tes accomplissements

Tu sais maintenant :
- ✅ Comprendre profondément comment fonctionnent les réseaux de neurones
- ✅ Implémenter un réseau from scratch (sans PyTorch/TensorFlow)
- ✅ Entraîner un modèle sur de vraies données
- ✅ Obtenir ~95%+ d'accuracy sur MNIST
- ✅ Optimiser et améliorer les performances
- ✅ Comprendre les concepts pour aborder le deep learning moderne

### 🚀 Prochaines Étapes

Maintenant que tu maîtrises les bases, tu peux:

1. **Approfondir les concepts**
   - Implémenter d'autres optimiseurs (RMSprop, AdaGrad)
   - Ajouter du dropout et batch normalization
   - Tester sur d'autres datasets (Fashion-MNIST, CIFAR-10)

2. **Passer aux frameworks modernes**
   - PyTorch ou TensorFlow
   - Réseaux convolutifs (CNN)
   - Transfer learning

3. **Projets pratiques**
   - Classification d'images personnalisées
   - Détection d'objets
   - Traitement du langage naturel

### 📚 Ressources pour continuer

- **Livres**:
  - "Deep Learning" - Ian Goodfellow
  - "Neural Networks and Deep Learning" - Michael Nielsen

- **Cours en ligne**:
  - Deep Learning Specialization (Coursera - Andrew Ng)
  - Fast.ai - Practical Deep Learning

- **Chaînes YouTube**:
  - 3Blue1Brown - Neural Networks series
  - Andrej Karpathy - Neural Networks: Zero to Hero

---

## 💪 Final Words

**Tu as parcouru un chemin incroyable !**

De la compréhension d'un simple neurone jusqu'à un réseau complet capable de reconnaître des chiffres manuscrits avec 95%+ de précision.

Tu n'as pas juste copié du code - tu **comprends vraiment** ce qui se passe sous le capot. C'est ça qui fait la différence !

Continue à apprendre, à expérimenter, et à construire des projets cool ! 🚀

---

**Bonne continuation dans ton aventure en Deep Learning ! 🎓🌟**